In [8]:
from dotenv import load_dotenv
import os
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage

load_dotenv()

#모델 설정
model = ChatOpenAI(model="gpt-4o-mini")

In [9]:
from langchain_openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.document_loaders import PyPDFLoader

# PDF 파일 로드. 파일의 경로 입력
loader = PyPDFLoader("231107_과실비율인정기준_온라인용.pdf")

# 페이지 별 문서 로드
docs = loader.load()

text_splitter = CharacterTextSplitter(
    separator="\n\n",
    chunk_size=100,
    chunk_overlap=10,
    length_function=len,
    is_separator_regex=False,
)

splits = text_splitter.split_documents(docs)

In [ ]:
import pdfplumber
import os
from langchain.schema import Document

# PDF에서 텍스트 추출
docs = []

# 폴더 내 파일 가져오기
path = 'pdf_files/'
file_list = os.listdir(path)

for doc_num, file_name in enumerate(file_list):
    file_path = os.path.join(path, file_name)
    try:
        extracted_text = ""  # PDF 한 파일의 전체 텍스트를 저장할 변수
        with pdfplumber.open(file_path) as pdf_file:
            for i, page in enumerate(pdf_file.pages):
                try:
                    # 텍스트 추출
                    text = page.extract_text()
                    if text:
                        extracted_text += text + "\n"  # 각 페이지의 텍스트를 합침
                except Exception as e:
                    print(f"페이지 {i + 1}에서 오류 발생: {e}")

        # 한 PDF 파일의 전체 텍스트를 하나의 Document로 저장
        if extracted_text.strip():  # 추출된 텍스트가 있으면 저장
            document = Document(
                page_content=extracted_text,
                metadata={"doc_number": doc_num + 1, "file_name": file_name}
            )
            docs.append(document)
        else:
            print(f"파일 '{file_name}'에서 텍스트 추출이 없습니다.")

    except Exception as e:
        print(f"파일 '{file_name}'에서 오류 발생: {e}")

# 텍스트 추출 결과 확인
print(f"총 {len(docs)}개의 PDF 파일에서 텍스트를 추출했습니다.")

파일 '2013가합15831_판결문_검수완료.pdf'에서 텍스트 추출이 없습니다.
총 125개의 PDF 파일에서 텍스트를 추출했습니다.


In [11]:
# 이미지로 처리되어 내용이 없는 파일 삭제
print(docs[0].page_content)

docs = docs[1:]

본 판결문은 판결서 인터넷열람 사이트에서 열람·출력되었습니다.
영리목적으로 이용하거나 무단 배포를 금합니다.게시일자 : 2016-10-07
본 판결문은 판결서 인터넷열람 사이트에서 열람·출력되었습니다.
영리목적으로 이용하거나 무단 배포를 금합니다.게시일자 : 2016-10-07
본 판결문은 판결서 인터넷열람 사이트에서 열람·출력되었습니다.
영리목적으로 이용하거나 무단 배포를 금합니다.게시일자 : 2016-10-07
본 판결문은 판결서 인터넷열람 사이트에서 열람·출력되었습니다.
영리목적으로 이용하거나 무단 배포를 금합니다.게시일자 : 2016-10-07
본 판결문은 판결서 인터넷열람 사이트에서 열람·출력되었습니다.
영리목적으로 이용하거나 무단 배포를 금합니다.게시일자 : 2016-10-07
본 판결문은 판결서 인터넷열람 사이트에서 열람·출력되었습니다.
영리목적으로 이용하거나 무단 배포를 금합니다.게시일자 : 2016-10-07
본 판결문은 판결서 인터넷열람 사이트에서 열람·출력되었습니다.
영리목적으로 이용하거나 무단 배포를 금합니다.게시일자 : 2016-10-07
본 판결문은 판결서 인터넷열람 사이트에서 열람·출력되었습니다.
영리목적으로 이용하거나 무단 배포를 금합니다.게시일자 : 2016-10-07
본 판결문은 판결서 인터넷열람 사이트에서 열람·출력되었습니다.
영리목적으로 이용하거나 무단 배포를 금합니다.게시일자 : 2016-10-07
본 판결문은 판결서 인터넷열람 사이트에서 열람·출력되었습니다.
영리목적으로 이용하거나 무단 배포를 금합니다.게시일자 : 2016-10-07
본 판결문은 판결서 인터넷열람 사이트에서 열람·출력되었습니다.
영리목적으로 이용하거나 무단 배포를 금합니다.게시일자 : 2016-10-07
본 판결문은 판결서 인터넷열람 사이트에서 열람·출력되었습니다.
영리목적으로 이용하거나 무단 배포를 금합니다.게시일자 : 2016-10-07



In [12]:
import json

# 과실 비율 데이터 읽어오기
# 폴더 내에 있는 파일 모두 가져오기
with open('accident_data_all_pages.json', 'r', encoding='utf-8') as f:
    file = json.load(f)


# JSON 데이터를 Document로 변환
def nested_json_to_documents(json_data):
    docs = []  # 문서 리스트 초기화
    # 중첩된 리스트를 순회하며 평탄화
    for entry in json_data:  # 최상위 리스트 순회
        content = (
            f"상황: {entry['상황']}\n"
            f"청구인 과실 비율: {entry['청구인 과실 비율']}\n"
            f"피청구인 과실 비율: {entry['피청구인 과실 비율']}\n"
        )
        docs.append(Document(page_content=content))  # Document 객체 추가
    return docs

docs_rate = nested_json_to_documents(file)

In [13]:
from langchain_core.prompts import ChatPromptTemplate

# 사고 상황 요약을 위한 프롬프트 템플릿 정의
summary_prompt = ChatPromptTemplate.from_messages([
    ('system', '주어진 문서 내의 "교통사고 발생 상황"을 "사고 원인"을 포함해서 한 문장으로 요약해줘. 단순 사고 상황에 대한 내용만 다루고 사건에 대한 판결의 내용은 넣지 말아줘.'),
    ('user', '{content}')
])

# 사고 상황을 요약하는 함수 (LLM 모델 사용)
def summarize_accident(accident_text):
    summary = summary_prompt.format_messages(content=accident_text)
    result = model.invoke(summary)
    return result  # 요약된 사고 상황 반환


# 문서 요약본 새 document로 저장
def summary_docs(original_doc, summary_text):
    updated_doc = Document(
        metadata={**original_doc.metadata, 'summary': summary_text},  # 요약 추가
        page_content=original_doc.page_content,
    )
    return updated_doc


# 요약된 문서들을 저장할 리스트
summarized_docs = []

# 모든 문서에 대해 요약 생성, 저장 및 임베딩
for i in range(len(docs)):
    summary = summarize_accident(docs[i].page_content)  # 요약 생성
    sum_doc = summary_docs(docs[i], summary)  # 요약 추가
    summarized_docs.append(sum_doc)  # 리스트에 추가

In [14]:
from langchain_openai import OpenAIEmbeddings
import faiss
from langchain_community.vectorstores import FAISS
from uuid import uuid4

# OpenAI 임베딩 모델 초기화
embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")

# 문서 별 고유 ID 생성
uuids = [f"page_{i+1}" for i in range(len(splits))]
uuids1 = [f"docs_{i+1}" for i in range(len(summarized_docs))]
uuids2 = [f"docs_{i+1}" for i in range(len(docs_rate))]


# FAISS 벡터 스토어 생성
vector_store_law = FAISS.from_documents(
    documents=splits, ids=uuids, embedding=embeddings)
vector_store_situation = FAISS.from_documents(
    documents=summarized_docs, ids=uuids1, embedding=embeddings)
vector_store_rate = FAISS.from_documents(
    documents=docs_rate, ids=uuids2, embedding=embeddings)

In [15]:
# FAISS 벡터 스토어 저장
SAVE_PATH1 = "vector_store_law"
SAVE_PATH2 = "vector_store_situation"
SAVE_PATH3 = "vector_store_rate"


# FAISS 저장
vector_store_law.save_local(SAVE_PATH1)
vector_store_situation.save_local(SAVE_PATH2)
vector_store_rate.save_local(SAVE_PATH3)